# Import Libraries and CSV File

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
import datetime as dt

In [56]:
# Read file in as Pandas dataframe

df = pd.read_csv('kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


# Model Approach

## My baseline and engineered models are built using grade as the primary feature instead of sqft_living due to the higher correlations sqft_living had with many other features in early analysis. This experiment is to evaluate whether sqft_living may be a better candidate once combined with engineered features 

# Preproccess and clean datatypes, null values, and value ranges

In [57]:
df['sqft_basement'] = pd.to_numeric(df['sqft_basement'], errors = 'coerce')

df['date'] = pd.to_datetime(df['date'], infer_datetime_format = True)

df['yrs_old'] = [i.year for i in df['date']] - df['yr_built']

df.drop(['date'], axis = 1, inplace = True)

In [58]:
df['view'] = df['view'].fillna(df['view'].mode()[0])

df['waterfront'] = df['waterfront'].fillna(df['waterfront'].mode()[0])

df['renovated'] = [0 if i == 0.0 else 1 for i in df['yr_renovated']]
df.drop(['yr_renovated'], axis = 1, inplace = True)

df['basement'] = [0 if i == 0.0 else 1 for i in df['sqft_basement']]
df.drop(['sqft_basement'], axis = 1, inplace = True)

In [59]:
df.drop(['yr_built', 'id'], axis = 1, inplace = True)

In [60]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,zipcode,lat,long,sqft_living15,sqft_lot15,yrs_old,renovated,basement
0,221900.0,3,1.00,1180,5650,1.0,0.0,0.0,3,7,1180,98178,47.5112,-122.257,1340,5650,59,0,0
1,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,98125,47.7210,-122.319,1690,7639,63,1,1
2,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,98028,47.7379,-122.233,2720,8062,82,1,0
3,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,98136,47.5208,-122.393,1360,5000,49,0,1
4,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,98074,47.6168,-122.045,1800,7503,28,0,0


In [61]:
df = df[(df['price'] >= 0) & (df['price'] <= 1000000)]
df = df[(df['bedrooms'] >= 0) & (df['bedrooms'] <= 7)]
df = df[(df['bathrooms'] >= 0) & (df['bathrooms'] <= 5)]

# Import distance to city feature

In [66]:
from haversine import haversine

seattle = [47.6092, -122.3363]
bellevue = [47.61555, -122.20392]

In [67]:
haversine(seattle, bellevue, unit ='mi')

6.181695095872431

In [68]:
df['lat_long'] = tuple(zip(df.lat, df.long))

In [69]:
seattle_distances = []
for i in df['lat_long']:
    seattle_distances.append(haversine((seattle), (i), unit = 'mi'))
df['distance_from_seattle'] = pd.Series(seattle_distances)

bellevue_distances = []
for i in df['lat_long']:
    bellevue_distances.append(haversine((bellevue), (i), unit = 'mi'))
df['distance_from_bellevue'] = pd.Series(bellevue_distances)

In [70]:
df['distance_from_city'] = df[['distance_from_bellevue', 'distance_from_seattle']].min(axis = 1)

In [71]:
df.drop(['lat', 'long', 'lat_long', 'distance_from_seattle', 'distance_from_bellevue'], axis = 1, inplace = True)

In [72]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,zipcode,sqft_living15,sqft_lot15,yrs_old,renovated,basement,distance_from_city
0,221900.0,3,1.00,1180,5650,1.0,0.0,0.0,3,7,1180,98178,1340,5650,59,0,0,7.622786
1,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,98125,1690,7639,63,1,1,7.766476
2,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,98028,2720,8062,82,1,0,8.561145
3,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,98136,1360,5000,49,0,1,6.655340
4,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,98074,1800,7503,28,0,0,7.402273


## Model with Distance to City Feature

In [48]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

outcome = 'price'
predictors = df.drop(['price'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.567
Model:                            OLS   Adj. R-squared:                  0.567
Method:                 Least Squares   F-statistic:                     1446.
Date:                Thu, 18 Feb 2021   Prob (F-statistic):               0.00
Time:                        14:46:52   Log-Likelihood:            -2.4762e+05
No. Observations:               18771   AIC:                         4.953e+05
Df Residuals:                   18753   BIC:                         4.954e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept          -1.191e+07   1.98e+06     -6.031      0.000   -1.58e+07   -8.04e+06
bedrooms           -1.303e+04   1415.908     -9.204      0.000   -1.58e+04   -1.03e+04
bathrooms           2.643e+04   2355.568     11.219      0.000    2.18e+04     3.1e+04
sqft_living           45.1424      4.545      9.932      0.000      36.233      54.051
sqft_lot               0.1522      0.033      4.627      0.000       0.088       0.217
floors              4.635e+04   2623.732     17.665      0.000    4.12e+04    5.15e+04
waterfront          1.035e+05   1.92e+04      5.385      0.000    6.58e+04    1.41e+05
view                1.593e+04   1646.942      9.671      0.000    1.27e+04    1.92e+04
condition           1.976e+04   1563.141     12.640      0.000    1.67e+04    2.28e+04
grade               8.537e+04   1542.276     55.350      0.000    8.23e+04    8.84e+04
sqft_above            10.2762      4.948      2.077      0.038       0.578      19.974
zipcode              114.5795     20.141      5.689      0.000      75.101     154.058
sqft_living15         64.0410      2.611     24.524      0.000      58.922      69.159
sqft_lot15            -0.2342      0.051     -4.637      0.000      -0.333      -0.135
yrs_old             2572.7238     47.977     53.625      0.000    2478.685    2666.762
renovated           1728.6000   2353.455      0.734      0.463   -2884.384    6341.584
basement            3.379e+04   3544.040      9.535      0.000    2.68e+04    4.07e+04
distance_from_city   -10.4750    151.897     -0.069      0.945    -308.207     287.257
==============================================================================
Omnibus:                      476.549   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              573.902
Skew:                           0.337   Prob(JB):                    2.39e-125
Kurtosis:                       3.530   Cond. No.                     2.10e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.1e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [50]:
# Drop column due to high p-value

df.drop(['distance_from_city'], axis = 1, inplace = True)

In [51]:
outcome = 'price'
predictors = df.drop(['price'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.567
Model:                            OLS   Adj. R-squared:                  0.567
Method:                 Least Squares   F-statistic:                     1646.
Date:                Thu, 18 Feb 2021   Prob (F-statistic):               0.00
Time:                        14:48:42   Log-Likelihood:            -2.6530e+05
No. Observations:               20116   AIC:                         5.306e+05
Df Residuals:                   20099   BIC:                         5.308e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -1.439e+07   1.92e+06     -7.499      0.000   -1.81e+07   -1.06e+07
bedrooms      -1.361e+04   1376.105     -9.892      0.000   -1.63e+04   -1.09e+04
bathrooms      2.536e+04   2264.360     11.200      0.000    2.09e+04    2.98e+04
sqft_living      46.1319      4.417     10.445      0.000      37.475      54.789
sqft_lot          0.1563      0.032      4.832      0.000       0.093       0.220
floors         4.979e+04   2436.079     20.441      0.000     4.5e+04    5.46e+04
waterfront      1.03e+05   1.91e+04      5.386      0.000    6.55e+04    1.41e+05
view           1.565e+04   1612.038      9.708      0.000    1.25e+04    1.88e+04
condition      1.989e+04   1554.384     12.794      0.000    1.68e+04    2.29e+04
grade          8.647e+04   1470.198     58.813      0.000    8.36e+04    8.93e+04
sqft_above       10.7462      4.761      2.257      0.024       1.415      20.078
zipcode         139.8211     19.564      7.147      0.000     101.475     178.168
sqft_living15    59.7966      2.491     24.001      0.000      54.913      64.680
sqft_lot15       -0.2213      0.049     -4.486      0.000      -0.318      -0.125
yrs_old        2470.2621     45.902     53.816      0.000    2380.291    2560.234
renovated      2811.9226   2268.219      1.240      0.215   -1633.972    7257.817
basement       3.563e+04   3395.207     10.495      0.000     2.9e+04    4.23e+04
==============================================================================
Omnibus:                      521.072   Durbin-Watson:                   1.964
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              629.514
Skew:                           0.340   Prob(JB):                    2.01e-137
Kurtosis:                       3.537   Cond. No.                     2.11e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.11e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [52]:
# drop column due to correlation with sqft features

df.drop(['grade'], axis = 1, inplace = True)

In [53]:
outcome = 'price'
predictors = df.drop(['price'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.492
Method:                 Least Squares   F-statistic:                     1301.
Date:                Thu, 18 Feb 2021   Prob (F-statistic):               0.00
Time:                        14:50:02   Log-Likelihood:            -2.6689e+05
No. Observations:               20116   AIC:                         5.338e+05
Df Residuals:                   20100   BIC:                         5.339e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -2.072e+07   2.07e+06     -9.991      0.000   -2.48e+07   -1.67e+07
bedrooms       -2.36e+04   1478.403    -15.961      0.000   -2.65e+04   -2.07e+04
bathrooms      3.421e+04   2446.002     13.985      0.000    2.94e+04     3.9e+04
sqft_living      63.8724      4.770     13.390      0.000      54.522      73.222
sqft_lot          0.1575      0.035      4.499      0.000       0.089       0.226
floors         6.707e+04   2618.076     25.618      0.000    6.19e+04    7.22e+04
waterfront      6.92e+04   2.07e+04      3.343      0.001    2.86e+04     1.1e+05
view           2.013e+04   1743.257     11.547      0.000    1.67e+04    2.35e+04
condition      2.142e+04   1682.551     12.731      0.000    1.81e+04    2.47e+04
sqft_above       47.6611      5.109      9.329      0.000      37.647      57.675
zipcode         209.2636     21.141      9.898      0.000     167.825     250.702
sqft_living15    97.7800      2.605     37.536      0.000      92.674     102.886
sqft_lot15       -0.3076      0.053     -5.764      0.000      -0.412      -0.203
yrs_old        1951.7820     48.769     40.021      0.000    1856.191    2047.373
renovated      4036.5300   2455.488      1.644      0.100    -776.428    8849.488
basement        5.53e+04   3657.805     15.118      0.000    4.81e+04    6.25e+04
==============================================================================
Omnibus:                      495.462   Durbin-Watson:                   1.964
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              538.181
Skew:                           0.378   Prob(JB):                    1.37e-117
Kurtosis:                       3.264   Cond. No.                     2.11e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.11e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [54]:
# drop column due to high p-value

df.drop(['renovated'], axis = 1, inplace = True)

In [55]:
outcome = 'price'
predictors = df.drop(['price'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.492
Method:                 Least Squares   F-statistic:                     1394.
Date:                Thu, 18 Feb 2021   Prob (F-statistic):               0.00
Time:                        14:50:39   Log-Likelihood:            -2.6689e+05
No. Observations:               20116   AIC:                         5.338e+05
Df Residuals:                   20101   BIC:                         5.339e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -2.069e+07   2.07e+06     -9.974      0.000   -2.48e+07   -1.66e+07
bedrooms      -2.365e+04   1478.083    -16.001      0.000   -2.65e+04   -2.08e+04
bathrooms      3.439e+04   2443.622     14.072      0.000    2.96e+04    3.92e+04
sqft_living      63.9091      4.770     13.397      0.000      54.559      73.259
sqft_lot          0.1569      0.035      4.482      0.000       0.088       0.226
floors         6.711e+04   2618.085     25.633      0.000     6.2e+04    7.22e+04
waterfront     6.957e+04   2.07e+04      3.361      0.001     2.9e+04     1.1e+05
view           2.016e+04   1743.219     11.566      0.000    1.67e+04    2.36e+04
condition      2.125e+04   1679.338     12.653      0.000     1.8e+04    2.45e+04
sqft_above       47.7082      5.109      9.338      0.000      37.694      57.723
zipcode         208.9113     21.141      9.882      0.000     167.473     250.350
sqft_living15    97.6744      2.604     37.505      0.000      92.570     102.779
sqft_lot15       -0.3072      0.053     -5.755      0.000      -0.412      -0.203
yrs_old        1960.9875     48.448     40.476      0.000    1866.025    2055.950
basement       5.528e+04   3657.933     15.111      0.000    4.81e+04    6.24e+04
==============================================================================
Omnibus:                      496.647   Durbin-Watson:                   1.963
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              539.646
Skew:                           0.379   Prob(JB):                    6.57e-118
Kurtosis:                       3.265   Cond. No.                     2.11e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.11e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [56]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [58]:
dfi = df.drop(['price'], axis = 1)

for i in range(len(dfi.columns[:-1])):
    v = vif(np.matrix(dfi[:-1]), i)
    print('Variance inflation factor for {}: {}'.format(dfi.columns[i], round(v, 2)))

Variance inflation factor for bedrooms: 26.25
Variance inflation factor for bathrooms: 28.41
Variance inflation factor for sqft_living: 102.34
Variance inflation factor for sqft_lot: 2.27
Variance inflation factor for floors: 16.77
Variance inflation factor for waterfront: 1.09
Variance inflation factor for view: 1.26
Variance inflation factor for condition: 34.16
Variance inflation factor for sqft_above: 90.16
Variance inflation factor for zipcode: 76.03
Variance inflation factor for sqft_living15: 27.31
Variance inflation factor for sqft_lot15: 2.48
Variance inflation factor for yrs_old: 6.1


### Even after columns with high p-values are dropped, there remains several columns with high vif due to the correlation between square footage and number of rooms

## Model with Zip to School District Conversion Feature

### Must skip executing model with distance to city cells to run successfully

In [62]:
def group_zips(z):
    
    if z in [98198, 98023, 98003, 98001, 98002, 98092]:
        return 'FederalWay_Auburn'
    
    elif z in [98032, 98031, 98030, 98042, 98058, 98178, 98056, 98055, 98146, 98166, 98148, 98188, 98038, 98022, 98010]:
        return 'Kent_Renton_Highline_Tahoma_Enumclaw'
    
    
    elif z in [98177, 98133, 98125, 98155, 98019, 98014]:
        return 'Shoreline_Riverview'
    
    elif z in [98024, 98065, 98045, 98070]:
        return 'Snoqualmie_Vashon'

    elif z in [98117, 98103, 98115, 98107, 98105, 98199, 98119, 98109, 98102, 
               98112, 98122, 98116, 98136, 98126,
               98106, 98144, 98108, 98118, 98028, 98011, 98072, 98077]:
        return 'Seattle_Northshore'
    
    elif z in [98034, 98052, 98053, 98033, 98074, 98059, 98027, 98075, 98029]:
        return 'LakeWashington_Issaquah'
    
    elif z in [98040]:
        return 'MercerIsland'
    
    elif z in [98039, 98004, 98005, 98007, 98008, 98006]:
        return 'Bellevue'
    
    elif z in [98168]:
        return 'Tukwila'

In [63]:
df['zipcode'] = df['zipcode'].map(group_zips)

In [64]:
df['zipcode'].head()

0    Kent_Renton_Highline_Tahoma_Enumclaw
1                     Shoreline_Riverview
2                      Seattle_Northshore
3                      Seattle_Northshore
4                 LakeWashington_Issaquah
Name: zipcode, dtype: object

## Do not execute code below. Use only for school district summary statistics

In [150]:
df['dpsf'] = df['price'] / df['sqft_living']

In [151]:
df['dpsf'].head()

0    188.050847
1    209.338521
2    233.766234
3    308.163265
4    303.571429
Name: dpsf, dtype: float64

In [158]:
df.sort_values(['price']).groupby('zipcode').mean()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,lat,long,sqft_living15,sqft_lot15,yrs_old,renovated,basement,dpsf
zipcode,,,,,,,,,,,,,,,,,,,
Bellevue,660437.058989,3.697566,2.161049,2223.596442,11614.596442,1.292603,0.000936,0.178839,3.691948,7.941011,1812.640449,47.592891,-122.149780,2246.306180,10888.213483,44.984082,0.193820,0.467228,312.528401
FederalWay_Auburn,290395.965394,3.376590,2.017303,1925.243257,14902.447328,1.376845,0.005089,0.181170,3.407125,7.408651,1710.157761,47.320145,-122.282209,1870.201018,13364.426972,36.364377,0.184224,0.313995,155.903659
Kent_Renton_Highline_Tahoma_Enumclaw,336981.576440,3.377578,2.024176,1927.878644,18922.798530,1.416212,0.004266,0.160465,3.438730,7.295805,1721.469543,47.414252,-122.169844,1871.308130,14792.037213,38.159279,0.202181,0.291064,182.352847
LakeWashington_Issaquah,581100.134263,3.519278,2.331214,2304.694011,15453.378452,1.600492,0.000273,0.085316,3.300246,8.044299,2093.466776,47.622079,-122.095738,2281.501504,13837.112387,27.529943,0.191687,0.298879,262.179017
MercerIsland,815539.904762,3.755102,2.234694,2388.612245,12203.034014,1.285714,0.000000,0.210884,3.836735,8.210884,1956.530612,47.561840,-122.225551,2588.775510,11456.965986,49.666667,0.204082,0.469388,359.968101
Seattle_Northshore,525258.529572,3.105111,1.909916,1753.696721,7460.962713,1.523947,0.000321,0.185471,3.415301,7.380906,1427.900836,47.637215,-122.314937,1696.753616,6821.848762,57.945677,0.224687,0.538251,320.096694
Shoreline_Riverview,429792.595824,3.224839,1.862286,1809.261777,16894.370450,1.364829,0.000000,0.148287,3.400964,7.252141,1527.662741,47.733198,-122.256843,1745.940578,15050.536938,48.674518,0.230728,0.400964,252.051580
Snoqualmie_Vashon,473009.198288,3.281027,2.230742,2199.485021,44187.266762,1.631241,0.025678,0.353780,3.209700,7.570613,2074.375178,47.497600,-121.933402,2118.325250,33092.248217,28.636234,0.195435,0.186876,226.695011
Tukwila,240328.371747,2.996283,1.433086,1468.624535,11269.843866,1.159851,0.000000,0.014870,3.245353,6.509294,1234.089219,47.489700,-122.307877,1471.680297,9744.022305,61.579926,0.193309,0.386617,175.372532


In [16]:
df.groupby('zipcode', as_index = False)['dpsf'].mean()

,zipcode,dpsf
0,Bellevue,312.528401
1,FederalWay/Auburn,155.903659
2,Kent/Renton/Highline/Tahoma/Enumclaw,182.352847
3,LakeWashington/Issaquah,262.179017
4,MercerIsland,359.968101
5,Seattle/Northshore,320.096694
6,Shoreline/Riverview,252.051580
7,Snoqualmie/Vashon,226.695011
8,Tukwila,175.372532


### One Hot Encode Zipcodes/School Districts

In [65]:
from sklearn.preprocessing import OneHotEncoder
ohc = OneHotEncoder()
ohe = ohc.fit_transform(df.zipcode.values.reshape(-1,1)).toarray()
dfOneHot = pd.DataFrame(ohe, columns = ['SD_' + str(ohc.categories_[0][i]) for i in range(len(ohc.categories_[0]))])
dfh = pd.concat([df, dfOneHot], axis = 1)
dfh.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,basement,SD_Bellevue,SD_FederalWay_Auburn,SD_Kent_Renton_Highline_Tahoma_Enumclaw,SD_LakeWashington_Issaquah,SD_MercerIsland,SD_Seattle_Northshore,SD_Shoreline_Riverview,SD_Snoqualmie_Vashon,SD_Tukwila
0,221900.0,3.0,1.00,1180.0,5650.0,1.0,0.0,0.0,3.0,7.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,538000.0,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,3.0,7.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,180000.0,2.0,1.00,770.0,10000.0,1.0,0.0,0.0,3.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,604000.0,4.0,3.00,1960.0,5000.0,1.0,0.0,0.0,5.0,7.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,510000.0,3.0,2.00,1680.0,8080.0,1.0,0.0,0.0,3.0,8.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [66]:
dfh.drop(['zipcode'], axis = 1, inplace = True)

In [67]:
dfh.isnull().sum()

price                                      1345
bedrooms                                   1345
bathrooms                                  1345
sqft_living                                1345
sqft_lot                                   1345
floors                                     1345
waterfront                                 1345
view                                       1345
condition                                  1345
grade                                      1345
sqft_above                                 1345
lat                                        1345
long                                       1345
sqft_living15                              1345
sqft_lot15                                 1345
yrs_old                                    1345
renovated                                  1345
basement                                   1345
SD_Bellevue                                1345
SD_FederalWay_Auburn                       1345
SD_Kent_Renton_Highline_Tahoma_Enumclaw 

In [68]:
# remove null values

dfh.dropna(inplace = True)

In [69]:
dfh.isnull().sum()

price                                      0
bedrooms                                   0
bathrooms                                  0
sqft_living                                0
sqft_lot                                   0
floors                                     0
waterfront                                 0
view                                       0
condition                                  0
grade                                      0
sqft_above                                 0
lat                                        0
long                                       0
sqft_living15                              0
sqft_lot15                                 0
yrs_old                                    0
renovated                                  0
basement                                   0
SD_Bellevue                                0
SD_FederalWay_Auburn                       0
SD_Kent_Renton_Highline_Tahoma_Enumclaw    0
SD_LakeWashington_Issaquah                 0
SD_MercerI

In [70]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [71]:
outcome = 'price'
predictors = dfh.drop(['price'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=dfh).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.690
Model:                            OLS   Adj. R-squared:                  0.690
Method:                 Least Squares   F-statistic:                     1669.
Date:                Sun, 21 Feb 2021   Prob (F-statistic):               0.00
Time:                        18:09:11   Log-Likelihood:            -2.4449e+05
No. Observations:               18771   AIC:                         4.890e+05
Df Residuals:                   18745   BIC:                         4.892e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                               -2.163e+07   8.06e+05    -26.823      0.000   -2.32e+07      -2e+07
bedrooms                                -8686.0738   1198.028     -7.250      0.000    -1.1e+04   -6337.831
bathrooms                                2.467e+04   1993.359     12.374      0.000    2.08e+04    2.86e+04
sqft_living                                40.1426      3.848     10.431      0.000      32.600      47.686
sqft_lot                                    0.2337      0.028      8.372      0.000       0.179       0.288
floors                                   2.977e+04   2226.845     13.369      0.000    2.54e+04    3.41e+04
waterfront                               1.449e+05   1.63e+04      8.897      0.000    1.13e+05    1.77e+05
view                                     2.661e+04   1396.014     19.062      0.000    2.39e+04    2.93e+04
condition                                2.689e+04   1316.577     20.422      0.000    2.43e+04    2.95e+04
grade                                    6.881e+04   1329.776     51.744      0.000    6.62e+04    7.14e+04
sqft_above                                 30.8939      4.205      7.347      0.000      22.652      39.136
lat                                      5.155e+05   5967.814     86.385      0.000    5.04e+05    5.27e+05
long                                     8345.8637   6987.426      1.194      0.232   -5350.124     2.2e+04
sqft_living15                              50.9271      2.226     22.880      0.000      46.564      55.290
sqft_lot15                                 -0.1300      0.043     -3.026      0.002      -0.214      -0.046
yrs_old                                  1866.2834     42.149     44.278      0.000    1783.667    1948.900
renovated                                2705.4432   1992.805      1.358      0.175   -1200.634    6611.521
basement                                 2.594e+04   3004.939      8.633      0.000    2.01e+04    3.18e+04
SD_Bellevue                             -2.403e+06   8.97e+04    -26.799      0.000   -2.58e+06   -2.23e+06
SD_FederalWay_Auburn                    -2.402e+06   8.97e+04    -26.797      0.000   -2.58e+06   -2.23e+06
SD_Kent_Renton_Highline_Tahoma_Enumclaw -2.408e+06   8.96e+04    -26.869      0.000   -2.58e+06   -2.23e+06
SD_LakeWashington_Issaquah              -2.405e+06   8.96e+04    -26.829      0.000   -2.58e+06   -2.23e+06
SD_MercerIsland                         -2.404e+06   8.99e+04    -26.729      0.000   -2.58e+06   -2.23e+06
SD_Seattle_Northshore                   -2.404e+06   8.96e+04    -26.825      0.000   -2.58e+06   -2.23e+06
SD_Shoreline_Riverview                  -2.402e+06   8.96e+04    -26.802      0.000   -2.58e+06   -2.23e+06
SD_Snoqualmie_Vashon                    -2.401e+06   8.97e+

In [72]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [73]:
dfhi = dfh.drop(['price'], axis = 1)

for i in range(len(dfhi.columns[:-1])):
    v = vif(np.matrix(dfhi[:-1]), i)
    print('Variance inflation factor for {}: {}'.format(dfhi.columns[i], round(v, 2)))

Variance inflation factor for bedrooms: 1.69
Variance inflation factor for bathrooms: 2.97
Variance inflation factor for sqft_living: 12.76
Variance inflation factor for sqft_lot: 2.01
Variance inflation factor for floors: 2.03
Variance inflation factor for waterfront: 1.1
Variance inflation factor for view: 1.21
Variance inflation factor for condition: 1.17
Variance inflation factor for grade: 2.81
Variance inflation factor for sqft_above: 13.31
Variance inflation factor for lat: 1.12
Variance inflation factor for long: 1.52
Variance inflation factor for sqft_living15: 2.69
Variance inflation factor for sqft_lot15: 2.05
Variance inflation factor for yrs_old: 2.19
Variance inflation factor for renovated: 1.02
Variance inflation factor for basement: 3.36
Variance inflation factor for SD_Bellevue: 66634.94
Variance inflation factor for SD_FederalWay_Auburn: 122288.65
Variance inflation factor for SD_Kent_Renton_Highline_Tahoma_Enumclaw: 262062.92
Variance inflation factor for SD_LakeWash

### # Drop columns one at a time and re run above model and vif after each column removal to decide if additional columns need to be removed

In [74]:
dfh.drop(['lat', 'long'], axis = 1, inplace = True)

In [75]:
dfh.drop(['renovated'], axis = 1, inplace = True)

In [76]:
dfh.drop(['grade', 'sqft_above'], axis = 1, inplace = True)

In [77]:
dfh.drop(['sqft_lot', 'sqft_living15', 'sqft_lot15'], axis = 1, inplace = True)

In [78]:
dfh.drop(['SD_Kent_Renton_Highline_Tahoma_Enumclaw'], axis = 1, inplace = True)

In [79]:
dfh.drop(['floors', 'condition'], axis = 1, inplace = True)

In [80]:
dfh.drop(['bathrooms'], axis = 1, inplace = True)

In [81]:
dfh.drop(['bedrooms'], axis = 1, inplace = True)

### Run final model once all vif are below 10

In [82]:
outcome = 'price'
predictors = dfh.drop(['price'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=dfh).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.411
Model:                            OLS   Adj. R-squared:                  0.410
Method:                 Least Squares   F-statistic:                     1006.
Date:                Sun, 21 Feb 2021   Prob (F-statistic):               0.00
Time:                        18:09:48   Log-Likelihood:            -2.5051e+05
No. Observations:               18771   AIC:                         5.011e+05
Df Residuals:                   18757   BIC:                         5.012e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                   5.554e+04   4919.569     11.290      0.000    4.59e+04    6.52e+04
sqft_living                  174.7790      1.684    103.801      0.000     171.479     178.079
waterfront                  6.554e+04   2.24e+04      2.927      0.003    2.16e+04    1.09e+05
view                        3.257e+04   1871.930     17.401      0.000    2.89e+04    3.62e+04
yrs_old                     1360.0567     43.912     30.972      0.000    1273.985    1446.128
basement                   -5189.8443   2389.527     -2.172      0.030   -9873.533    -506.155
SD_Bellevue                  566.2493   5358.556      0.106      0.916   -9937.005    1.11e+04
SD_FederalWay_Auburn        4914.2754   4276.064      1.149      0.250   -3467.198    1.33e+04
SD_LakeWashington_Issaquah  6116.0971   3540.274      1.728      0.084    -823.160    1.31e+04
SD_MercerIsland             1.266e+04    1.3e+04      0.972      0.331   -1.29e+04    3.82e+04
SD_Seattle_Northshore       5356.6584   3125.345      1.714      0.087    -769.300    1.15e+04
SD_Shoreline_Riverview      5147.0195   4350.054      1.183      0.237   -3379.479    1.37e+04
SD_Snoqualmie_Vashon        1.011e+04   6430.793      1.573      0.116   -2492.402    2.27e+04
SD_Tukwila                  1.418e+04   9852.367      1.439      0.150   -5131.759    3.35e+04
==============================================================================
Omnibus:                      360.739   Durbin-Watson:                   1.951
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              381.680
Skew:                           0.349   Prob(JB):                     1.32e-83
Kurtosis:                       2.993   Cond. No.                     4.21e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.21e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [83]:
dfhi = dfh.drop(['price'], axis = 1)

for i in range(len(dfhi.columns[:-1])):
    v = vif(np.matrix(dfhi[:-1]), i)
    print('Variance inflation factor for {}: {}'.format(dfhi.columns[i], round(v, 2)))

Variance inflation factor for sqft_living: 3.98
Variance inflation factor for waterfront: 1.09
Variance inflation factor for view: 1.21
Variance inflation factor for yrs_old: 2.72
Variance inflation factor for basement: 1.84
Variance inflation factor for SD_Bellevue: 1.19
Variance inflation factor for SD_FederalWay_Auburn: 1.35
Variance inflation factor for SD_LakeWashington_Issaquah: 1.66
Variance inflation factor for SD_MercerIsland: 1.03
Variance inflation factor for SD_Seattle_Northshore: 2.12
Variance inflation factor for SD_Shoreline_Riverview: 1.33
Variance inflation factor for SD_Snoqualmie_Vashon: 1.12


# Model Summary

## Once grade and other variables with low significance and high vif begin to be removed the r-squared deteriorates to 0.411. My determination is that sqft_living is not a suitable feature to build the model around due to the nature of similarity with many other features. There is no need to validate this model because it will not be retained for use. 